<a href="https://colab.research.google.com/github/dmbk/Anomaly-Detection-System/blob/master/VidasaExtended_Conv3DLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install keras-layer-normalization
!pip install imageio

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from PIL import Image
from tensorflow.keras import layers
import time

from IPython import display

from os.path import join
from os import listdir
from os.path import isfile, join, isdir

#import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2DTranspose, Conv3DTranspose, ConvLSTM2D, BatchNormalization, LayerNormalization, TimeDistributed, Conv2D, Conv3D, MaxPool3D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential, load_model

import matplotlib.pyplot as plt
import argparse
from os.path import dirname
import cv2
from google.colab.patches import cv2_imshow

TensorFlow 2.x selected.


In [0]:
class Config:
    def __init__(self, data_dir):
        self.DATASET_PATH = join(data_dir,"UCSDped1/Train")
        self.SINGLE_TEST_PATH = join(data_dir,"UCSDped1/Test/Test010")
        self.TEST_DIR = join(data_dir,"UCSDped1/Test/")
        self.BATCH_SIZE = 4
        self.EPOCHS = 3
        self.MODEL_PATH = join(data_dir,"model_conv3d_lstm.hdf5")

conf = Config(data_dir="/content/drive/My Drive/UCSD_Anomaly_Dataset.v1p2/") 
physical_devices = tf.config.list_physical_devices('GPU') 
try: 
  tf.config.experimental.set_memory_growth(physical_devices[0], True) 
except: 
  # Invalid device or cannot modify virtual devices once initialized. 
  pass

In [6]:
def get_model(reload_model=False):

    if reload_model:
        return load_model(conf.MODEL_PATH,custom_objects={'LayerNormalization': LayerNormalization})
    seq = Sequential()
    seq.add(TimeDistributed(Conv3D(128, (11, 11, 11), strides=4, padding="same"), batch_input_shape=(None, 10, 32, 256, 256, 1)))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv3D(64, (5, 5, 5), strides=2, padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv3D(32, (3, 3, 3), strides=1, padding="same")))
    seq.add(LayerNormalization())
    # # # # #
    seq.add(TimeDistributed(ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True)))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(ConvLSTM2D(32, (3, 3), padding="same", return_sequences=True)))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True)))
    seq.add(LayerNormalization())
    # # # # #
    seq.add(TimeDistributed(Conv3DTranspose(32, (3, 3, 3), strides=1, padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv3DTranspose(64, (5, 5, 5), strides=2, padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv3DTranspose(128, (11, 11, 11), strides=4, padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv3D(1, (11, 11, 11), activation="sigmoid", padding="same")))
    print(seq.summary())
    return seq

model =  get_model()

def get_loss(true_clips, pred_clips):
    return tf.keras.losses.MSE(y_true=true_clips, y_pred=pred_clips)

def get_optimizer():
    return tf.optimizers.Adam(1e-4, decay=1e-5, epsilon=1e-6)
    
optimizer = get_optimizer()

@tf.function
def train_step(images):    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        reconstructed_clips = model(clips, training=True)
        reconstruction_loss = get_loss(reconstructed_clips, clips)

    gradients_of_model = gen_tape.gradient(reconstruction_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients_of_model, model.trainable_variables))
   

#get_model()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_20 (TimeDis (None, 10, 8, 64, 64, 128 170496    
_________________________________________________________________
layer_normalization_18 (Laye (None, 10, 8, 64, 64, 128 256       
_________________________________________________________________
time_distributed_21 (TimeDis (None, 10, 4, 32, 32, 64) 1024064   
_________________________________________________________________
layer_normalization_19 (Laye (None, 10, 4, 32, 32, 64) 128       
_________________________________________________________________
time_distributed_22 (TimeDis (None, 10, 4, 32, 32, 32) 55328     
_________________________________________________________________
layer_normalization_20 (Laye (None, 10, 4, 32, 32, 32) 64        
_________________________________________________________________
time_distributed_23 (TimeDis (None, 10, 4, 32, 32, 64)